In [1]:
import csv
import codecs
import re
import xml.etree.cElementTree as ET
import pprint

In [2]:
OSMFILE = 'austin_texas.osm'

In [3]:
# Count the number of each tag types

def count_tags(filename):    
    tags = {}
    tree = ET.parse(filename)
    for child in tree.iter():
        if child.tag in tags.keys():
            tags[child.tag] += 1
        else:
            tags[child.tag] = 1        
    pprint.pprint(tags)
    
count_tags(OSMFILE)

{'bounds': 1,
 'member': 9086,
 'nd': 812361,
 'node': 736203,
 'osm': 1,
 'relation': 961,
 'tag': 346079,
 'way': 82267}


In [4]:
# Count the number of unique users that have contributed to this area

def unique_users(filename):
    users = set()
    for _,element in ET.iterparse(filename):
        if 'user' in element.attrib:
            users.add(element.attrib['user'])
    pprint.pprint(len(users))
    
unique_users(OSMFILE)

438


In [5]:
# Check if street types are in correct format and update if they are not

from collections import defaultdict

street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)


expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons"]

# UPDATE THIS VARIABLE
mapping = { "St": "Street",
            "St.": "Street",
            "Ave": "Avenue",
            "Ave.": "Avenue",
            "Rd": "Road",
            "Rd.": "Road"}

def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)

def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")

def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types

def update_name(name, mapping):
            
    prefix_name_type = name.split(' ')
    length = len(prefix_name_type)
    prefix_name = prefix_name_type[0:length-1]
    type = prefix_name_type[length-1]

    if type in mapping.keys():
        new_type = mapping[type]
        name = ' '.join(prefix_name) + ' ' + new_type
    
    return name

def test():
    st_types = audit(OSMFILE)
    #pprint.pprint(dict(st_types))

    for st_type, ways in st_types.iteritems():
        for name in ways:
            better_name = update_name(name, mapping)
            print name, "=>", better_name

test()

Camino Alto => Camino Alto
Verde Vista => Verde Vista
Desert Flower => Desert Flower
Honeycomb Ridge => Honeycomb Ridge
Timberline Ridge => Timberline Ridge
Mission Ridge => Mission Ridge
Burnet Road #600 => Burnet Road #600
Via Media => Via Media
E 38th 1/2 St. => E 38th 1/2 Street
E. 43rd St. => E. 43rd Street
Spurlock Valley => Spurlock Valley
N Interstate 35 Frontage Rd => N Interstate 35 Frontage Road
Barton Springs Rd => Barton Springs Road
West 35th Street Cutoff => West 35th Street Cutoff
Rue de St Germaine => Rue de St Germaine
Cromwell Hill => Cromwell Hill
Island Way => Island Way
Almarion Way => Almarion Way
Cactus Wren Way => Cactus Wren Way
Verdant Way => Verdant Way
Rock Way => Rock Way
Arnulfo Alonso Way => Arnulfo Alonso Way
Sandra Muraida Way => Sandra Muraida Way
Bold Ruler Way => Bold Ruler Way
Stamford Way => Stamford Way
Heritage Way => Heritage Way
Airole Way => Airole Way
Woodcutters Way => Woodcutters Way
Wild Basin Lodge => Wild Basin Lodge
Loma Linda => Loma 

In [6]:
import codecs


NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')


# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']




def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements

    if element.tag == 'node':
        for attr in NODE_FIELDS:
            node_attribs[attr] = element.attrib[attr]
            
        element_id = element.attrib['id']
        for child in element.iter():
            if child.tag == 'tag':
                sub_tag = {}
                sub_tag['id'] = element_id
                sub_tag['value'] = child.attrib['v']
                if re.search(PROBLEMCHARS, child.attrib['k']):
                    break
                else:
                    if re.search(LOWER_COLON, child.attrib['k']):
                        sub_tag['key'] = ':'.join(child.attrib['k'].split(':')[1:])
                        sub_tag['type'] = child.attrib['k'].split(':')[0]
                    else:
                        sub_tag['key'] = child.attrib['k']
                        sub_tag['type'] = default_tag_type
                tags.append(sub_tag)
        
        return {'node': node_attribs, 'node_tags': tags}
    elif element.tag == 'way':
        
        for attr in WAY_FIELDS:
            way_attribs[attr] = element.attrib[attr]
            
        element_id = element.attrib['id']
        nd_position = 0
        for child in element.iter():
            if child.tag == 'nd':
                sub_nd = {}
                sub_nd['id'] = element_id
                sub_nd['node_id'] = child.attrib['ref']
                sub_nd['position'] = nd_position
                nd_position += 1
                way_nodes.append(sub_nd)
            
            if child.tag == 'tag':
                sub_tag = {}
                sub_tag['id'] = element_id
                sub_tag['value'] = child.attrib['v']
                if re.search(PROBLEMCHARS, child.attrib['k']):
                    break
                else:
                    if re.search(LOWER_COLON, child.attrib['k']):
                        sub_tag['key'] = ':'.join(child.attrib['k'].split(':')[1:])
                        sub_tag['type'] = child.attrib['k'].split(':')[0]
                    else:
                        sub_tag['key'] = child.attrib['k']
                        sub_tag['type'] = default_tag_type
                tags.append(sub_tag)

        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}


# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()
            

class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)
            
# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        #validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                #if validate is True:
                #    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])
            
            
            
            
            
process_map(OSMFILE, validate=False)

In [8]:
import sqlite3
import csv

con=sqlite3.connect('austin_texas.db')
cursor = con.cursor()


#cursor.execute("""DROP TABLE nodes;""")
#cursor.execute("""DROP TABLE nodes_tags;""")
#cursor.execute("""DROP TABLE ways; """)
#cursor.execute("""DROP TABLE ways_tags;""")
#cursor.execute("""DROP TABLE ways_nodes;""")

nodes_query = """CREATE TABLE nodes (
    id INTEGER PRIMARY KEY NOT NULL,
    lat REAL,
    lon REAL,
    user TEXT,
    uid INTEGER,
    version INTEGER,
    changeset INTEGER,
    timestamp TEXT
);"""

nodes_tags_query = """CREATE TABLE nodes_tags (
    id INTEGER,
    key TEXT,
    value TEXT,
    type TEXT,
    FOREIGN KEY (id) REFERENCES nodes(id)
);"""


ways_query = """CREATE TABLE ways (
    id INTEGER PRIMARY KEY NOT NULL,
    user TEXT,
    uid INTEGER,
    version TEXT,
    changeset INTEGER,
    timestamp TEXT
);"""


ways_tags_query = """CREATE TABLE ways_tags (
    id INTEGER NOT NULL,
    key TEXT NOT NULL,
    value TEXT NOT NULL,
    type TEXT,
    FOREIGN KEY (id) REFERENCES ways(id)
);"""


ways_nodes_query = """CREATE TABLE ways_nodes (
    id INTEGER NOT NULL,
    node_id INTEGER NOT NULL,
    position INTEGER NOT NULL,
    FOREIGN KEY (id) REFERENCES ways(id),
    FOREIGN KEY (node_id) REFERENCES nodes(id)
);"""

cursor.execute(nodes_query)
cursor.execute(nodes_tags_query)
cursor.execute(ways_query)
cursor.execute(ways_tags_query)
cursor.execute(ways_nodes_query)


def UnicodeDictReader(utf8_data, **kwargs):
    csv_reader = csv.DictReader(utf8_data, **kwargs)
    for row in csv_reader:
        yield {key: unicode(value, 'utf-8') for key, value in row.iteritems()}


# Write nodes table from nodes.csv
with open('nodes.csv') as csvfile:
    myCSVReader = UnicodeDictReader(csvfile)
    for row in myCSVReader:
        sql = "INSERT INTO nodes (id,lat,lon,user,uid,version,changeset,timestamp) VALUES (?,?,?,?,?,?,?,?);"
        data = (row['id'],row['lat'],row['lon'],row['user'],row['uid'],row['version'],row['changeset'],row['timestamp'])
        cursor.execute(sql, data)
    con.commit()



# Write nodes_tags table from nodes_tags.csv
with open('nodes_tags.csv') as csvfile:
    myCSVReader = UnicodeDictReader(csvfile)
    for row in myCSVReader:
        sql = "INSERT INTO nodes_tags (id,key,value,type) VALUES (?,?,?,?);"
        data = (row['id'],row['key'],row['value'],row['type'])
        cursor.execute(sql, data)
    con.commit()


# Write ways table from ways.csv
with open('ways.csv') as csvfile:
    myCSVReader = UnicodeDictReader(csvfile)
    for row in myCSVReader:
        sql = "INSERT INTO ways (id,user,uid,version,changeset,timestamp) VALUES (?,?,?,?,?,?);"
        data = (row['id'],row['user'],row['uid'],row['version'],row['changeset'],row['timestamp'])
        cursor.execute(sql, data)
    con.commit()


# Write ways_tags table from ways_tags.csv
with open('ways_tags.csv') as csvfile:
    myCSVReader = UnicodeDictReader(csvfile)
    for row in myCSVReader:
        sql = "INSERT INTO ways_tags (id,key,value,type) VALUES (?,?,?,?);"
        data = (row['id'],row['key'],row['value'],row['type'])
        cursor.execute(sql, data)
    con.commit()


# Write ways_nodes table from ways_nodes.csv
with open('ways_nodes.csv') as csvfile:
    myCSVReader = UnicodeDictReader(csvfile)
    for row in myCSVReader:
        sql = "INSERT INTO ways_nodes (id,node_id,position) VALUES (?,?,?);"
        data = (row['id'],row['node_id'],row['position'])
        cursor.execute(sql, data)
    con.commit()

con.close()